<a href="https://colab.research.google.com/github/jorgik1/ai-site-generator/blob/main/site_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is a Python script designed to generate a visually appealing and responsive portfolio website for a given GitHub user. The script utilizes OpenAI's API to generate the HTML content, which is then deployed to Vercel, a popular platform for hosting static websites. The script fetches the user's information from the GitHub API, and the resulting portfolio website highlights the user's profile details in a well-organized and mobile-friendly layout.

Key features of the project include:

1.   Fetching user information from the GitHub API.
2.   Generating a custom HTML layout using OpenAI's API.
3.   Deploying the generated website to Vercel.

By leveraging OpenAI's API and Vercel's deployment platform, the script provides an automated solution for creating professional, personalized, and visually stunning portfolio websites for GitHub users.

In [ ]:
!pip install openai python-dotenv langchain

In [ ]:
!curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
!sudo apt-get install -y nodejs
!npm install -g vercel

In [ ]:
import os
import subprocess

import openai
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from google.colab import files
uploaded = files.upload()
load_dotenv("./.env")
VERCEL_TOKEN = os.environ.get("VERCEL_TOKEN")
openai.api_key = os.environ.get("OPENAI_API_KEY")


vercel_website_name = "developer-portfolio"

import requests
import tempfile
from pathlib import Path
import json

def deployVercel(html_content: str, project_name: str, vercel_token: str):
    with tempfile.TemporaryDirectory() as temp_dir:
        project_dir = Path(temp_dir) / project_name
        os.makedirs(project_dir)

        # Write the HTML content to the index.html file
        with open(project_dir / "index.html", "w") as f:
            f.write(html_content)

        # Write the vercel.json configuration file
        vercel_config = {
            "name": project_name,
            "version": 2,
            "builds": [{"src": "index.html", "use": "@vercel/static"}],
        }

        with open(project_dir / "vercel.json", "w") as f:
            json.dump(vercel_config, f)

        # Deploy to Vercel using the CLI
        cmd = ["vercel", "--token", vercel_token, "-y", "--prod"]

        result = subprocess.run(cmd, cwd=project_dir, capture_output=True, text=True)

        if result.returncode == 0:
            print("Deployment READY!!")
            print(f"Final URL: {result.stdout.strip()}")
        else:
            print("Deployment failed:")
            print(result.stderr)

def create_website(github_json):
    prompt = f"""Make a beautiful website. Provide only HTML and related code. Your code should not only be visually stunning, but also intuitive and mobile friendly. Include some animations where appropriate. 
             Remember to prioritize readability and organization, and to create a polished final product. Include the following information from this json in the website, 
             but skip the ones that end in url. Do not provide any other text or answer: 
            ---
            {github_json}"""

    chat = ChatOpenAI(temperature=0.2, max_tokens=2500)

    messages = [
        SystemMessage(content="You are a AI senior frontend designer and developer trained by OpenAI. Generate HTML that is colorful and well designed. Use gradient and animations where applicable."),
        HumanMessage(content=prompt),
    ]

    response = chat(messages)

    reply = response.content
    return reply

def createPortfolio(user):
    response = requests.get(f"https://api.github.com/users/{user}")
    if response.status_code != 200:
        print(f"Error: Unable to fetch the page. Status code: {response.status_code}")
        return
    print(response.json())

    html_content = create_website(response.json())
    deployVercel(html_content, vercel_website_name, VERCEL_TOKEN)

user = input("What developer github ID to make personal website for?")

createPortfolio(user)